In [15]:
import numpy as np
import pandas as pd
import nltk
#nltk.download('punkt') # one time execution
import re


In [25]:
df = pd.read_csv("tweetsinfo.csv",encoding= 'unicode_escape')
df.head()

,no,twt
0,1,Reliance Infrastructure Limited Stock Analysis...
1,2,Which 4 stock bucket would u choose for long t...
2,3,Companies Working On 5G Technologies In India...
3,4,More recently we've seen the negative impacts ...
4,5,seeing the recent FB future growth reliance ch...


In [27]:
df['twt'][2]

' Companies Working On 5G Technologies In India - Reliance Ind - Bharti Airtel - Tejas Network- Tech Mahindra - TCS - Siti Neâ\x80¦'

In [28]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in df['twt']:
  sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list

In [29]:
sentences[:5]

['Reliance Infrastructure Limited Stock Analysis Report  Target 217  298 - 07.02.2022',
 'Which 4 stock bucket would u choose for long term -A - TCS HDFC Ltd Dabur Asian Paints B - Infosys Kotak Mahindra B',
 ' Companies Working On 5G Technologies In India - Reliance Ind - Bharti Airtel - Tejas Network- Tech Mahindra - TCS - Siti Neâ\x80¦',
 "More recently we've seen the negative impacts to businesses that have a reliance on the ad model (reference Meta's biggest ever stock market loss recently).",
 'It is slowly becoming clear people value their privacy (9/11)']

In [30]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip glove*.zip
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()
len(word_embeddings)

400000

In [31]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

#nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

<ipython-input-31-fd4b868df624>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")


In [32]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [33]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [34]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

from sklearn.metrics.pairwise import cosine_similarity

for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [35]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [37]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

# Extract top 10 sentences as the summary
for i in range(3):
  print(ranked_sentences[i][1])

More recently we've seen the negative impacts to businesses that have a reliance on the ad model (reference Meta's biggest ever stock market loss recently).
Reliance Infrastructure Limited Stock Analysis Report  Target 217  298 - 07.02.2022
seeing the recent FB future growth reliance charts totally shows how investors see stock as nothing but a shallow statistical model
